In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
AI_STUDIO_KEY = os.getenv("GOOGLE_AI_STUDIO_KEY")

## Connect


In [ ]:
import weaviate

# Connect to the local instance deployed with Docker Compose
client = weaviate.connect_to_local(
    headers={
       "X-Google-Api-Key": AI_STUDIO_KEY,
    }
)

client.is_ready()

In [ ]:
import weaviate, os

client = weaviate.connect_to_wcs(
    cluster_url=os.getenv("WORKSHOP_DEMO_URL"),
    auth_credentials=weaviate.auth.AuthApiKey(os.getenv("WORKSHOP_DEMO_KEY_ADMIN")),
    headers={
       "X-Palm-Api-Key": AI_STUDIO_KEY,
    }
)

client.is_ready()

## Create collection with a gerative model

In [ ]:
from weaviate.classes.config import Configure, Property, DataType

if (client.collections.exists("Papers")):
    client.collections.delete("Papers")

client.collections.create(
    "Papers",
    vectorizer_config=Configure.Vectorizer.text2vec_palm(
        model_id="embedding-001",

        project_id="-", # this needs fixing
        api_endpoint="generativelanguage.googleapis.com",
    ),
    
    generative_config=Configure.Generative.palm(
        model_id="gemini-pro",
        api_endpoint="generativelanguage.googleapis.com",
        project_id="-",
    ),

    properties=[  # Define properties (Optional)
        Property(name="chunk", data_type=DataType.TEXT),
        Property(name="arxiv_id", data_type=DataType.TEXT, skip_vectorization=True),
        Property(name="title", data_type=DataType.TEXT, skip_vectorization=True),
        Property(name="url", data_type=DataType.TEXT, skip_vectorization=True),
        Property(name="chunk_no", data_type=DataType.INT),
    ],
)

## Close the client when done

In [ ]:
client.close()